In [ ]:
pip install accelerate transformers SentencePiece  bitsandbytes einops xformers openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "openaccess-ai-collective/minotaur-13b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
    )

In [ ]:
import pandas as pd

df = pd.read_json('dummy.json')

df

,n.documentName,n.pageContent,ID(n)
0,Lead Paint Law: Information for Owners and Sel...,<p>Maryland and federal laws about lead paint ...,100
1,Baltimore City Rent Escrow Law,<p>The purpose of the rent escrow law is to pr...,101
2,Cosigning a Lease,<p>There are times when a tenant needs a cosig...,111
3,When is Refusal-to-Rent Illegal Housing Discri...,<p>People often wrongly assume that any discri...,112
4,Anne Arundel County Rental and Housing Laws,\r\n <h2>Topics on this page:</h2>\r\n \r\n ...,749
...,...,...,...
452,Contesting a Will (Caveat Proceeding),<p>Contesting a will is a formal way of asking...,23350
453,Time limit for filing a will with the Register...,"\r\n \r\n <p>After someone dies, Maryland la...",23351
454,Youth Health Law,<h2>Topics on this page:</h2>\r\n \r\n <ul>\...,23352
455,Introduction to Juvenile Record Expungement,<p>Maryland law permits a person with a juveni...,23957


In [ ]:
df.shape

(457, 3)

In [ ]:
from bs4 import BeautifulSoup
import re
import string

#removing html code
def remove_html(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    return text
df["n.pageContent"] = df["n.pageContent"].apply(remove_html)


#Create a pipeline to test out different questions and answers

In [ ]:
def return_questions(questions_str):
  ques_arr = []
  var1 = questions_str.split("1. ")
  var2 =  var1[1].split("2. ")
  ques1 = var2[0]
  ques_arr.append(ques1)
  var3 = var2[1].split("3. ")
  ques2 = var3[0]
  ques_arr.append(ques2)
  ques3 = var3[1]
  ques_arr.append(ques3)
  print("ques_arr is: ", ques_arr)
  return ques_arr

In [ ]:
def return_answers(question_str, content):
    prompt=f"""
              Instruction : If the user question is not related to the Document content,make sure to answer with 'No related content found'.
              Do not answer the question based on your own knowledge. Do not repeat yourself in the answer.
              Do not start the answer with 'The user question is related to the Document content' . Provide the answer in bullet points where appropriate.
              Follow these 2 steps exactly:
              1. Read the user provided Document content below and aggregrate the data
              Document content : """ + big_content + """
              2. Answer the following question in-depth using only the above context.
              User Question: """ + question_str + """

              Answer:
    """
    # actual_prompt = """
    #               Instruction :
    #           Follow these 2 steps exactly:
    #           1. Read the user provided Content below and aggregrate the data
    #           Content : """ + content + """
    #           . 2. Answer the following question using only the above context. If the user question is not related to the Content,make sure to answer with 'No related content found'.
    #           User Question: """ + question_str + """
    #           Answer :
    #           """
    # get the start time
    st = time.time()

    sequences = pipeline(
        prompt,
        max_length=2000,
        top_k=1, #If you set k to 1, the model will only pick the top token
        early_stopping = True,
        # num_return_sequences=1,
        # no_repeat_ngram_size = 5,
        return_full_text=False,
        temperature = 0
    )

    gen_output = ""
    for seq in sequences:
        # print(f"{seq['generated_text']}")
        gen_output = gen_output + seq['generated_text']
    print("gen output for ques: ",question_str, " is: ", gen_output)

    # get the end time
    et = time.time()

    # get the execution time
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')

    torch.cuda.empty_cache()
    return gen_output, elapsed_time

In [ ]:
import os
import openai
import time

def get_code_score(text):
  time.sleep(2)
  openai.api_key = "<Enter Open API KEY>"
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k-0613",
    messages=[
      {"role": "user", "content": text}
    ]
  )
  response = completion.choices[0].message
  response_content = response['content']
  buffer_arr = response_content.split("-")
  score, reason = buffer_arr[0], buffer_arr[1]
  print("score is: ", score)
  print("reason is: ", reason)
  return score, reason

In [ ]:
import time
import torch

#We want to store: Content, question, answer, time taken to create the answer
question_arr = []
answer_arr = []
time_arr = []
content_arr = []
score_arr = []
reason_arr = []

for i in range(0, 40):
    print("i is: ", i)
    content = df["n.pageContent"][i]
    torch.cuda.empty_cache()
    big_content = content[:5000]

    instruction = "Give me 3 in-depth questions that can be answered using the given content. These questions should have a high cosine similarity with the content.  "

    input_text = " The content is: " + big_content + " ." + instruction + " The 3 questions are:  "

    sequences = pipeline(
        input_text,
        max_length=2000,
        top_k=5,
        # num_return_sequences=1,
        # no_repeat_ngram_size = 5,
        # early_stopping = True,
        return_full_text=False
    )

    gen_output = ""
    for seq in sequences:
        # print(f"{seq['generated_text']}")
        gen_output = gen_output + seq['generated_text']
    print("gen output for questions is: ", gen_output)
    ques_arr = return_questions(gen_output)

    for ques in ques_arr:
      time.sleep(3)
      answer, elapsed_time = return_answers(ques, big_content)
      # print("Answer is: ", answer)
      if answer.strip() == "":
        print("Inside empty answer, try again")
        # while answer.strip() == "":
        #   time.sleep(3)
        #   answer,elapsed_time = return_answers(ques, big_content)
        continue
      elif elapsed_time < 2:
        time.sleep(3)
        print("Inside time_elapsed < 2")
        answer, elapsed_time = return_answers(ques, big_content)
        print("Updated time elapsed: ", elapsed_time)
        continue
      generated_answer = answer.strip()
      question_arr.append(ques)
      answer_arr.append(generated_answer)
      time_arr.append(elapsed_time)
      content_arr.append(big_content)
      instruction = "Please rate the answer to the given question based on the content from 1-10, with 1 being the lowest. Return the score as (score - reason of score). If the question is answered with information from outside the given content, set the score to 0."
      gpt_string = "content is: " + big_content + " question : " + ques + " answer: "+ generated_answer + instruction
      print("ques: ", ques)
      print("answer: ", generated_answer)
      score, reason = get_code_score(gpt_string)
      score_arr.append(score)
      reason_arr.append(reason)
      time.sleep(5)
      torch.cuda.empty_cache()


i is:  0
gen output for questions is:  1. What is the definition of an Affected Property under Maryland’s lead paint law?  2. What are the duties of landlords and property owners of residential housing built before 1978?  3. What are the requirements for renovation and repair activities on pre-1978 residential properties?
ques_arr is:  ['What is the definition of an Affected Property under Maryland’s lead paint law?  ', 'What are the duties of landlords and property owners of residential housing built before 1978?  ', 'What are the requirements for renovation and repair activities on pre-1978 residential properties?']
gen output for ques:  What is the definition of an Affected Property under Maryland’s lead paint law?    is:  
              An Affected Property is a pre-1978 residential rental property/unit (“Affected Properties”) that is regulated by Maryland’s Reduction of Lead Risk in Housing Act. Owner-occupied properties/units are not regulated. Affected Properties that are Maryla


KeyboardInterrupt



In [ ]:
question_arr_series = pd.Series(question_arr)
answer_arr_series = pd.Series(answer_arr)
time_arr_series = pd.Series(time_arr)
content_arr_series = pd.Series(content_arr)
score_arr_series = pd.Series(score_arr)
reason_arr_series = pd.Series(reason_arr)

final_df = pd.concat([content_arr_series,question_arr_series, answer_arr_series, time_arr_series, score_arr_series, reason_arr_series], axis = 1)

In [ ]:
final_df.columns = ['Content', 'Question', 'Answer', 'Execution Time', 'Score', 'Reason']
final_df

,Content,Question,Answer,Execution Time,Score,Reason
0,Maryland and federal laws about lead paint are...,What is the definition of an Affected Property...,An Affected Property is a pre-1978 residential...,9.546767,8,The answer provides the accurate definition o...
1,Maryland and federal laws about lead paint are...,What are the duties of landlords and property ...,The duties of landlords and property owners of...,13.588029,8,The answer provides a thorough and accurate s...
2,Maryland and federal laws about lead paint are...,What are the requirements for renovation and r...,The requirements for renovation and repair act...,14.602602,7,The answer provides a thorough and accurate s...
3,The purpose of the rent escrow law is to prote...,What is the purpose of the rent escrow law in ...,The purpose of the rent escrow law in Baltimor...,7.732678,8,The answer provides a clear and accurate expl...
4,The purpose of the rent escrow law is to prote...,What are the conditions that must exist for a ...,The rent escrow process can be used by a tenan...,36.965873,7,The content provides a clear explanation of t...
...,...,...,...,...,...,...
107,Topics on this page Smoke Alarms General Req...,What are the penalties for not installing or m...,The penalties for not installing or maintainin...,8.833796,8,The answer directly quotes the relevant infor...
108,Topics on this page: Property conditions and ...,What is the definition of a serious or dangero...,"The serious or dangerous conditions include, b...",12.790264,NaN,NaN
109,Ending a Lease because of a Duty Order If a s...,What is the maximum amount of rent a service m...,The maximum amount of rent a service member ca...,4.530972,NaN,NaN
110,Ending a Lease because of a Duty Order If a s...,What is the formula used to calculate the maxi...,The formula used to calculate the maximum rent...,3.808231,NaN,NaN


In [ ]:
final_df.to_csv("output_final_with_scores_for_minotaur.csv")

In [ ]:
content = df["n.pageContent"][0]
content

"Maryland and federal laws about lead paint are designed to reduce childhood lead poisoning. This article describes the duties of landlords and property owners of residential housing built before 1978. Information for renters and buyers can be found in another article.   Topics on this page:  Which homes are covered by Maryland’s lead paint law? Use of lead paint Owning and selling an Affected Property Landlord's duties Frequently asked questions for landlords    Which homes are covered by Maryland’s lead paint law? Only pre-1978 residential rental properties/units (“Affected Properties”) are regulated by Maryland’s Reduction of Lead Risk in Housing Act. Owner-occupied properties/units are not regulated. Affected Properties that are Maryland Department of the Environment (MDE) certified lead-free or limited lead-free are exempt from Maryland’s lead paint law. Affected Properties owned or operated by federal, State, or local government or by a public, quasi-public, or municipal corporat

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
big_content = content[:7000]
check_big_content_len = big_content.split(" ")
len(check_big_content_len)

1099

In [ ]:
big_content

"Maryland and federal laws about lead paint are designed to reduce childhood lead poisoning. This article describes the duties of landlords and property owners of residential housing built before 1978. Information for renters and buyers can be found in another article.   Topics on this page:  Which homes are covered by Maryland’s lead paint law? Use of lead paint Owning and selling an Affected Property Landlord's duties Frequently asked questions for landlords    Which homes are covered by Maryland’s lead paint law? Only pre-1978 residential rental properties/units (“Affected Properties”) are regulated by Maryland’s Reduction of Lead Risk in Housing Act. Owner-occupied properties/units are not regulated. Affected Properties that are Maryland Department of the Environment (MDE) certified lead-free or limited lead-free are exempt from Maryland’s lead paint law. Affected Properties owned or operated by federal, State, or local government or by a public, quasi-public, or municipal corporat

In [ ]:
prompt=f"""
        Instruction :
        Follow these 2 steps exactly:
        1. Read the user provided Content below and aggregrate the data
        Content : """ + big_content + """
        . 2. Answer the following question using only the above context. If the user question is not related to the Content,make sure to answer with 'No related content found'.
        User Question:  What is marijuana?
        Answer :
"""


In [ ]:
import time

# get the start time
st = time.time()

sequences = pipeline(
    prompt,
    max_length=4000,
    top_k=1, #If you set k to 1, the model will only pick the top token
    early_stopping = True,
    # num_return_sequences=1,
    return_full_text=False,
    temperature = 0
)

for seq in sequences:
    print(f"{seq['generated_text']}")

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


        No related content found.
Execution time: 0.8632216453552246 seconds


In [ ]:
# get the start time
st = time.time()

instruction = "Give me 3 in-depth questions based on the given content. These questions should have a high cosine similarity with the content.  "

input_text = " The content is: " + big_content + " ." + instruction + " The 3 questions are:  "

sequences = pipeline(
    input_text,
    max_length=2000,
    top_k=5,
    # num_return_sequences=1,
    # no_repeat_ngram_size = 5,
    # early_stopping = True,
    return_full_text=False
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
torch.cuda.empty_cache()

Result: 1. What is the definition of an Affected Property under Maryland's lead paint law?  2. What are the duties of landlords and property owners of residential housing built before 1978?  3. What are the requirements for renovation and repair activities on pre-1978 residential properties?
Execution time: 4.584323406219482 seconds
